In [ ]:
import re
import tabula
import requests
import json
import csv
import boto3
import pandas as pd
import numpy as np
from data_extraction import DataExtractor as de
from database_utils import DatabaseConnector as dc
from datetime import datetime as dt

extr = de()
conn = dc()

In [ ]:
df = extr.read_rds_table(conn, conn.list_db_tables()[1])
df = df.replace({'NULL': np.nan})
df.dropna(how='all', subset=['email_address', 'address', 'phone_number'], inplace=True) # Drop people that we cannot contact
df = df[~df['email_address'].str.contains('@') == False]
df['country_code'] = df['country_code'].replace({'GGB':'GB'})
df.reset_index(drop=True, inplace=True)
for i in range(len(df)):
    try:
        df['date_of_birth'][i] = dt.strptime(df['date_of_birth'][i], '%B %Y %d').date()
    except ValueError:
        try:
            df['date_of_birth'][i] = dt.strptime(df['date_of_birth'][i], '%Y/%m/%d').date()
        except ValueError:
            try:
                df['date_of_birth'][i] = dt.strptime(df['date_of_birth'][i], '%Y %B %d').date()
            except ValueError:
                pass

In [ ]:
dg = df[df.loc[:, 'country_code'] == 'GB']
dg['phone_number'].replace(to_replace=')', value='', inplace=True)
df.update(dg)
display(dg)